### ====================================
### OrdinalEncoder (Ordinal=順序)
sklearn.preprocessing.OrdinalEncoder
[▼DOC](
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html
)  
category_encoders.OrdinalEncoder
[▼DOC](
http://contrib.scikit-learn.org/category_encoders/ordinal.html
)  
> この変換器への入力は  
カテゴリ的（離散的）特徴量によって  
取られる値を表す**整数または文字列の配列**のようなものでなければなりません．  
特徴量は，**順序の整数**に変換されます．  
これにより，特徴量ごとに1列の整数（0からn_categories - 1）が得られます．

・**category_encoders**のOrdinalEncoder。  
　**sklearn.preprocessing**にもあるけど、こっちのほうが使いやすいか。  
・元々数値の場合は変換されないのかな？  
・数値のマッピング1,2,,,は、もとの値の登場順に振っている模様。  
・データフレーム丸ごと変換できる。  
・nanがあっても変換できる。

In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format 

In [2]:
# どこの所属？
try: 
    from sklearn.preprocessing import OrdinalEncoder
    print("sklearn.preprocessingにある")
except ImportError:
    print("sklearn.preprocessingにはない")

try:
    from category_encoders import OrdinalEncoder
    print("category_encodersにある")
except ImportError:
    print("category_encodersにはない")

sklearn.preprocessingにある
category_encodersにある


In [3]:
from category_encoders import OrdinalEncoder

In [4]:
df = pd.DataFrame({
        "COLOR" : ["Red", "Yellow", "Red", "Red", "Blue"],
        "SIZE" : ["S", "M", "L", "S", "S"],
        "WEIGHT" : [200, 100, 300, 300, 300],
        "HARDNESS" : ["hard", "soft", "medium", "hard", "soft"],
        "OUTCOME" : [1,2,0,0,0] })
df

,COLOR,SIZE,WEIGHT,HARDNESS,OUTCOME
0,Red,S,200,hard,1
1,Yellow,M,100,soft,2
2,Red,L,300,medium,0
3,Red,S,300,hard,0
4,Blue,S,300,soft,0


In [5]:
# 説明変数X と 目的変数Yに分ける
df_X = df[["COLOR", "SIZE", "WEIGHT","HARDNESS"]].copy()
df_Y = df[["OUTCOME"]].copy()

In [6]:
df_X_test = pd.DataFrame({
        "COLOR" : ["Red", "Yellow", "Red", "Green"],
        "SIZE" : ["S", "M", "L", "XL", ],
        "WEIGHT" : [200, 100, 300, 400],
        "HARDNESS" : ["hard", "soft", "medium", "maxhard"] })
df_X_test

,COLOR,SIZE,WEIGHT,HARDNESS
0,Red,S,200,hard
1,Yellow,M,100,soft
2,Red,L,300,medium
3,Green,XL,400,maxhard


- エンコードしてみる

In [11]:
print("エンコード前")
display(df_X)

print("エンコード後")
oe = OrdinalEncoder()
df_X_oe = oe.fit_transform(df_X)
display(df_X_oe)

エンコード前


,COLOR,SIZE,WEIGHT,HARDNESS
0,Red,S,200,hard
1,Yellow,M,100,soft
2,Red,L,300,medium
3,Red,S,300,hard
4,Blue,S,300,soft


エンコード後


,COLOR,SIZE,WEIGHT,HARDNESS
0,1,1,200,1
1,2,2,100,2
2,1,3,300,3
3,1,1,300,1
4,3,1,300,2


- 並び順の指定

In [8]:
# 数値のマッピング1,2,,,は、もとの値の登場順に振っているっぽい。
# ので、自分で指定したい場合は、順序のリストを渡すといい。
print("エンコード前")
display(df_X[["HARDNESS"]])

# 都度、新しくインスタンスを作成して
# 自分が指定した順序を覚えさせて、変換する。
# 複数列まとめて指定する方法もあるんだろうけどわからなかった。。。
oe = OrdinalEncoder()
order = [['soft'], ['medium'], ['hard']] # この順で、1,2,3,,を振る
oe.fit(order)
df_X_le = oe.transform(np.ravel(df_X[["HARDNESS"]]))

print("エンコード後")
df_X_le.columns = ["HARDNESS"]
display(df_X_le)

エンコード前


,HARDNESS
0,hard
1,soft
2,medium
3,hard
4,soft


エンコード後


,HARDNESS
0,3
1,1
2,2
3,3
4,1


- Nanがあっても変更できる

In [9]:
# Nanがあっても変換できる
df_X_tmp = pd.DataFrame({
        "COLOR" : ["Red", "Yellow", "Red", np.nan, "Blue"],
        "HARDNESS" : ["hard", np.nan, "medium", "hard", "soft"] })

#print("エンコード前")
display(df_X_tmp)

print("エンコード後")
oe = OrdinalEncoder()
oe.fit_transform(df_X_tmp)

,COLOR,HARDNESS
0,Red,hard
1,Yellow,NaN
2,Red,medium
3,NaN,hard
4,Blue,soft


エンコード後


,COLOR,HARDNESS
0,1,1
1,2,2
2,1,3
3,3,1
4,4,4


- fit時にはなかった値があっても変換できる

In [10]:
# fit時にはなかった値があっても変換できる
print("エンコード前")
display(df_X_test)

print("エンコード後")
oe = OrdinalEncoder()
oe.fit(df_X)
oe.transform(df_X_test)

エンコード前


,COLOR,SIZE,WEIGHT,HARDNESS
0,Red,S,200,hard
1,Yellow,M,100,soft
2,Red,L,300,medium
3,Green,XL,400,maxhard


エンコード後


,COLOR,SIZE,WEIGHT,HARDNESS
0,1.00,1.00,200,1.00
1,2.00,2.00,100,2.00
2,1.00,3.00,300,3.00
3,-1.00,-1.00,400,-1.00


- お試し

In [24]:
print("エンコード前")
display(df_X)

print("エンコード後")
oe = OrdinalEncoder(
    #verbose=1,
    cols=["COLOR","SIZE"],
    mapping=[
        {"col":"COLOR", "mapping": {"Red":5, "Yellow":10, "Blue":20}},
        {"col":"SIZE" , "mapping": {"S":500, "M":1000, "L":2000}}
    ],
    handle_unknown = -1,
)
df_X_oe = oe.fit_transform(df_X)
display(df_X_oe)

エンコード前


,COLOR,SIZE,WEIGHT,HARDNESS
0,Red,S,200,hard
1,Yellow,M,100,soft
2,Red,L,300,medium
3,Red,S,300,hard
4,Blue,S,300,soft


エンコード後


,COLOR,SIZE,WEIGHT,HARDNESS
0,5,500,200,hard
1,10,1000,100,soft
2,5,2000,300,medium
3,5,500,300,hard
4,20,500,300,soft
